In [2]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow import keras
print(tf.__version__)

2.8.0


In [4]:
data_dir = r'./data/bird_photos'
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,  #分割数据集
    subset="training",          #数据集类型
    seed=123,
    image_size=(224, 224),
    batch_size=32)

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,  #分割数据集
    subset="validation",          #数据集类型
    seed=123,
    image_size=(224, 224),
    batch_size=32)

Found 565 files belonging to 4 classes.
Using 452 files for training.


2023-02-24 12:48:03.395695: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-24 12:48:04.786069: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 10249 MB memory:  -> device: 0, name: NVIDIA RTX A2000 12GB, pci bus id: 0000:04:00.0, compute capability: 8.6


Found 565 files belonging to 4 classes.
Using 113 files for validation.


In [5]:
class BottleNeck(keras.Model):
    def __init__(self, growth_rate, bn_size = 4, dropout = 0.3):
        super().__init__()
        self.bn1 = layers.BatchNormalization()
        self.relu = layers.Activation("relu"),
        self.conv1 = layers.Conv2D(filters=bn_size * growth_rate, kernel_size=(1, 1), 
                                   strides=1, padding='same')
        self.bn2 = layers.BatchNormalization()
        self.conv2 = layers.Conv2D(filters=growth_rate, kernel_size=(3, 3), 
                                   strides=1, padding='same')
        self.dropout = layers.Dropout(rate=dropout)

        self.listLayers = [
            self.bn1,
            self.relu,
            self.conv1,
            self.bn2,
            self.relu,
            self.conv2,
            self.dropout
            ]
        
    def call(self, x):
        tem = x
        for layer in self.listLayers.layers:
            x = layer(x)
        return layers.concatenate([tem, x], axis=-1)



class Transition(tf.keras.Model):
    def __init__(self, growth_rate):
        super().__init__()
        self.bn1 = layers.BatchNormalization()
        self.relu = layers.Activation('relu')
        self.conv1 = layers.Conv2D(filters = growth_rate, kernel_size=(1, 1),
                                   strides = 1, activation = 'relu', padding='same')
        self.pooling = layers.AveragePooling2D(pool_size=(2,2), strides = 2,padding='same')

        self.listLayers = [
            self.bn1,
            self.relu,
            self.conv1,
            self.pooling
        ]
    def call(self,x):
        for layer in self.listLayers.layers:
            x = layer(x)
        return x

class DenseBlock(tf.keras.Model):
    def __init__(self, num_layer, growth_rate, bn_size = 4, dropout = 0.3, efficient=False):
        super().__init__()
        self.efficient = efficient
        self.listLayers = []
        if self.efficient:
            _x = tf.recompute_grad(BottleNeck(growth_rate, bn_size = bn_size, dropout = dropout))
        else:_x =BottleNeck(growth_rate, bn_size = bn_size, dropout = dropout)
        for _ in range(num_layer):
            self.listLayers.append(BottleNeck(growth_rate, bn_size = bn_size, dropout = dropout))
    
    def call(self, x):
        for layer in self.listLayers.layers:
            x = layer(x)
        return x


class DenseNet(tf.keras.Model):
    def __init__(self, num_init_feature, growth_rate, block_config, num_classes, 
                 bn_size=4, dropout=0.3, compression_rate=0.5, efficient=False):
        super().__init__()
        self.num_channels = num_init_feature
        self.conv = layers.Conv2D(filters = num_init_feature, kernel_size=7,
                                strides = 2, padding='same')
        self.bn = layers.BatchNormalization()
        self.relu = layers.Activation('relu')
        self.max_pool = layers.MaxPool2D(pool_size=3, strides=2, padding='same')

        self.dense_block_layers = []
        for i in block_config[:-1]:
            self.dense_block_layers.append( DenseBlock(num_layer =i, growth_rate = growth_rate,
                                                 bn_size = bn_size, dropout = dropout, efficient=efficient))
            self.num_channels = compression_rate * (self.num_channels + growth_rate * i)
            self.dense_block_layers.append( Transition(self.num_channels))
            
        self.dense_block_layers.append( DenseBlock(num_layer =block_config[-1], growth_rate = growth_rate,
                                             bn_size = bn_size, dropout = dropout, efficient=efficient))
        
        self.avgpool = layers.GlobalAveragePooling2D()
        self.fc = tf.keras.layers.Dense(units=num_classes, activation=tf.keras.activations.softmax)
            
    def call(self,x):
        x = self.conv(x)
        x = self.bn(x)
        x = self.relu(x)
        x = self.max_pool(x)
        
        for layer in self.dense_block_layers.layers:
            x = layer(x)

        
        x = self.avgpool(x)
        return self.fc(x)

In [6]:
model = DenseNet(num_init_feature=64,
                 growth_rate=32,
                 block_config=[6,12,24,16],
                 compression_rate=0.5,
                 num_classes = 4,
                 dropout=0.0,
                 efficient=True)

x = tf.random.normal((1,224,224,3))
for layer in model.layers:
    x = layer(x)
    print(layer.name, 'output shape:\t', x.shape)

2023-02-24 12:49:30.781500: I tensorflow/stream_executor/cuda/cuda_dnn.cc:368] Loaded cuDNN version 8101


conv2d output shape:	 (1, 112, 112, 64)
batch_normalization output shape:	 (1, 112, 112, 64)
activation output shape:	 (1, 112, 112, 64)
max_pooling2d output shape:	 (1, 56, 56, 64)


2023-02-24 12:49:34.190024: I tensorflow/stream_executor/cuda/cuda_blas.cc:1786] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


dense_block output shape:	 (1, 56, 56, 256)
transition output shape:	 (1, 28, 28, 128)
dense_block_1 output shape:	 (1, 28, 28, 512)
transition_1 output shape:	 (1, 14, 14, 256)
dense_block_2 output shape:	 (1, 14, 14, 1024)
transition_2 output shape:	 (1, 7, 7, 512)
dense_block_3 output shape:	 (1, 7, 7, 1024)
global_average_pooling2d output shape:	 (1, 1024)
dense output shape:	 (1, 4)


In [7]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [10]:
opt = tf.keras.optimizers.Adam(learning_rate=0.002,decay=0.01)

model.compile(optimizer=opt,
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [11]:
epochs = 40
history = model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=epochs
)

Epoch 1/40


/root/miniconda3/envs/myconda/lib/python3.9/site-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


15/15 [==============================] - 29s 753ms/step - loss: 2.5196 - accuracy: 0.4801 - val_loss: 96574.7109 - val_accuracy: 0.2301
Epoch 2/40
15/15 [==============================] - 5s 327ms/step - loss: 1.2141 - accuracy: 0.6217 - val_loss: 6833.5015 - val_accuracy: 0.3186
Epoch 3/40
15/15 [==============================] - 5s 314ms/step - loss: 0.6587 - accuracy: 0.7367 - val_loss: 672.4532 - val_accuracy: 0.3186
Epoch 4/40
15/15 [==============================] - 5s 314ms/step - loss: 0.5117 - accuracy: 0.8252 - val_loss: 76.6003 - val_accuracy: 0.3186
Epoch 5/40
15/15 [==============================] - 5s 316ms/step - loss: 0.5910 - accuracy: 0.8009 - val_loss: 31.9198 - val_accuracy: 0.3186
Epoch 6/40
15/15 [==============================] - 5s 316ms/step - loss: 0.4022 - accuracy: 0.8650 - val_loss: 7.1209 - val_accuracy: 0.3097
Epoch 7/40
15/15 [==============================] - 5s 315ms/step - loss: 0.2295 - accuracy: 0.9181 - val_loss: 2.9968 - val_accuracy: 0.3628
Epoch